In [1]:
import numpy as np
#from scipy import stats
#import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import boto3
import pandas as pd

In [2]:
# #Fetching data from staging bucket
# s3 = s3fs.S3FileSystem()
# fs = s3fs.core.S3FileSystem()
 
# #s3_path = "gbs-gl-predictive-staging-dev/gl-predictive-analytics/are=9462/gl-transactions/*.parquet" # old link
# s3_path_txn = "gbs-gl-predictive-staging-dev/gl-predictive-analytics/gl-transactions/company_code=9462/*.parquet"
# #s3_path_gl = "gbs-gl-predictive-staging-dev/gl-predictive-analytics/gl-ending-balance/company_code=9462/*.parquet"
# txn_paths_from_s3 = fs.glob(path=s3_path_txn)
# #gl_paths_from_s3 = fs.glob(path=s3_path_gl)

# myopen = s3.open
# #use s3fs as the filesystem
# fp_obj_txn = fp.ParquetFile(txn_paths_from_s3,open_with=myopen)
# #fp_obj_gl = fp.ParquetFile(gl_paths_from_s3,open_with=myopen)

In [3]:
# fp_obj_txn
# # # #fp_obj_gl

In [4]:
# #taking only the neccessary columns and converting it into pandas
# data_txn = fp_obj_txn.to_pandas(['gl_account', 'posting_calendar_date', 'balance_sheet_account', 'entry_calendar_date', 'entered_by', 'debit_credit_indicator','Amt_loc_Currency'])
# # data_txn.shape

In [5]:
# data_txn.head(50)

# Saving Subset data to S3 bucket for use

In [6]:
#     import s3fs
#     extra_args = {
#         "ServerSideEncryption": "aws:kms"
#     }
#     s3=s3fs.S3FileSystem(anon=False,s3_additional_kwargs=extra_args)

In [7]:
# with s3.open('s3://gbs-gl-predictive-staging-dev/gl-predictive-analytics/stats/are=9462/end_balances/data_txn.csv1',mode='w',newline = '\n') as f:
#     data_txn.to_csv(f,sep=",",index=False,encoding="utf-8")

In [8]:
# data_txn.to_csv('data_txn.csv',sep=",",index=False,encoding="utf-8")

# Read CSV file 

In [9]:
# data_txn = pd.read_csv('s3://gbs-gl-predictive-staging-dev/gl-predictive-analytics/stats/are=9462/end_balances/data_txn.csv1')
data_txn = pd.read_csv('/home/ec2-user/SageMaker/data-science-ending-balance/code/data-science/02-preprocessing/data_txn.csv')
data_txn.head()

,gl_account,posting_calendar_date,balance_sheet_account,entry_calendar_date,entered_by,debit_credit_indicator,Amt_loc_Currency
0,0069614010,2019-01-15,NaN,2019-01-15,Z0009EIA,S,479.12
1,0043060000,2019-01-11,X,2019-01-11,SYS_BIZTALK,S,22.20
2,0060101000,2019-01-24,NaN,2019-01-24,Z0009ETH,S,131.52
3,0020201000,2019-01-04,X,2019-01-04,Z0009ETH,H,-2.33
4,0048110000,2019-01-19,X,2019-01-19,SBSBATCHVERA,S,168.00


In [10]:
# pip list

In [11]:
data_txn.dtypes

gl_account                 object
posting_calendar_date      object
balance_sheet_account      object
entry_calendar_date        object
entered_by                 object
debit_credit_indicator     object
Amt_loc_Currency          float64
dtype: object

In [12]:
# list_posting_dates = data_txn.posting_calendar_date.unique()
# # list_gl_ac_post_excl = df_AC_post_exclusions.gl_account.unique()
# list_posting_dates

In [13]:
# list_entry_dates = data_txn.entry_calendar_date.unique()
# # list_gl_ac_post_excl = df_AC_post_exclusions.gl_account.unique()
# list_entry_dates

In [14]:
# converting text to dates wehre required
data_txn["entry_calendar_date"] = pd.to_datetime(data_txn["entry_calendar_date"]) # ParserError: Unknown string format: USD
data_txn["posting_calendar_date"] = pd.to_datetime(data_txn["posting_calendar_date"]) # not likely a string

In [15]:
data_txn.dtypes

gl_account                        object
posting_calendar_date     datetime64[ns]
balance_sheet_account             object
entry_calendar_date       datetime64[ns]
entered_by                        object
debit_credit_indicator            object
Amt_loc_Currency                 float64
dtype: object

In [16]:
data_txn.shape

(9705113, 7)

# Difference of months between posting and entry dates

In [17]:
#data_example = data_txn.loc[data_txn["gl_account"] =="0067593000"]

#### Flag transactions that were entered later 

In [18]:
# convert entry_date and posting date to entry month and posting month 
import datetime
data_txn['month_of_posting'] = data_txn['posting_calendar_date'] + pd.offsets.MonthEnd(0) 
data_txn['month_of_entry'] = data_txn['entry_calendar_date']  + pd.offsets.MonthEnd(0) 
data_txn['delay_cnt'] = data_txn['month_of_entry'] - data_txn['month_of_posting'] 
data_txn.head(5)

,gl_account,posting_calendar_date,balance_sheet_account,entry_calendar_date,entered_by,debit_credit_indicator,Amt_loc_Currency,month_of_posting,month_of_entry,delay_cnt
0,0069614010,2019-01-15,NaN,2019-01-15,Z0009EIA,S,479.12,2019-01-31,2019-01-31,0 days
1,0043060000,2019-01-11,X,2019-01-11,SYS_BIZTALK,S,22.20,2019-01-31,2019-01-31,0 days
2,0060101000,2019-01-24,NaN,2019-01-24,Z0009ETH,S,131.52,2019-01-31,2019-01-31,0 days
3,0020201000,2019-01-04,X,2019-01-04,Z0009ETH,H,-2.33,2019-01-31,2019-01-31,0 days
4,0048110000,2019-01-19,X,2019-01-19,SBSBATCHVERA,S,168.00,2019-01-31,2019-01-31,0 days


In [19]:
data_txn['late_entry_flg'] = np.where(data_txn['month_of_entry']> data_txn['month_of_posting'], 1, 0)
data_txn[['entry_calendar_date', 'posting_calendar_date', 'delay_cnt', 'late_entry_flg']].head(5)

,entry_calendar_date,posting_calendar_date,delay_cnt,late_entry_flg
0,2019-01-15,2019-01-15,0 days,0
1,2019-01-11,2019-01-11,0 days,0
2,2019-01-24,2019-01-24,0 days,0
3,2019-01-04,2019-01-04,0 days,0
4,2019-01-19,2019-01-19,0 days,0


In [20]:
data_txn.shape

(9705113, 11)

In [21]:
data_txn[data_txn['late_entry_flg']==1]

,gl_account,posting_calendar_date,balance_sheet_account,entry_calendar_date,entered_by,debit_credit_indicator,Amt_loc_Currency,month_of_posting,month_of_entry,delay_cnt,late_entry_flg
515,0061000050,2019-01-13,NaN,2019-02-05,SYS_BIZTALK,S,717.60,2019-01-31,2019-02-28,28 days,1
858,0043051000,2019-01-21,X,2019-02-04,SYS_BIZTALK,S,904.16,2019-01-31,2019-02-28,28 days,1
1176,0043051000,2019-01-03,X,2019-02-04,Z003XMKK,S,646.34,2019-01-31,2019-02-28,28 days,1
1622,0061000050,2019-01-24,NaN,2019-02-04,SYS_BIZTALK,S,296.08,2019-01-31,2019-02-28,28 days,1
1714,0043060000,2019-01-09,X,2019-02-04,SYS_BIZTALK,S,222.06,2019-01-31,2019-02-28,28 days,1
...,...,...,...,...,...,...,...,...,...,...,...
9636105,0043060000,2020-01-24,X,2020-02-03,SYS_BIZTALK,H,-607.12,2020-01-31,2020-02-29,29 days,1
9643820,0043051000,2020-01-24,X,2020-02-03,SYS_BIZTALK,S,607.12,2020-01-31,2020-02-29,29 days,1
9646244,0043060000,2020-01-23,X,2020-02-03,SYS_BIZTALK,H,-367.85,2020-01-31,2020-02-29,29 days,1
9659092,0061790000,2020-01-26,NaN,2020-02-03,Z002CJ8V,S,145.83,2020-01-31,2020-02-29,29 days,1


### Number of accounts with late entry flags

In [22]:
#data_txn['delay_cnt']
cnt_bl_ac = pd.crosstab(index=data_txn["late_entry_flg"].fillna('missing'),  # Make a crosstab
                      columns="count") 
cnt_bl_ac

MemoryError: Unable to allocate 148. MiB for an array with shape (2, 9705113) and data type int64

# Amount and count of transactions - before and after closing

In [ ]:
# get total of debit, credit and net transactions 
txn_dr_cr = data_txn.pivot_table(index=['gl_account', 'month_of_posting', 'month_of_entry'],
                                    columns='debit_credit_indicator', values=['Amt_loc_Currency'] ,  aggfunc=[np.sum, 'count']) # reshape data 
txn_dr_cr = pd.DataFrame(txn_dr_cr.to_records())
txn_dr_cr.tail(5)

In [ ]:
txn_dr_cr.columns = (['gl_account', 'month_of_posting', 'month_of_entry', 'txn_cr_amt',  'txn_dr_amt', 'txn_cr_cnt',  'txn_dr_cnt' ])
txn_dr_cr['txn_net_amt'] =txn_dr_cr[["txn_dr_amt", "txn_cr_amt"]].sum(axis=1)
txn_dr_cr['txn_tot_cnt'] =txn_dr_cr[["txn_dr_cnt", "txn_cr_cnt"]].sum(axis=1)
txn_dr_cr['txn_abs_amt'] =abs(txn_dr_cr[["txn_dr_amt", "txn_cr_amt"]]).sum(axis=1)
txn_dr_cr['late_entry_flg'] = np.where(txn_dr_cr['month_of_entry']> txn_dr_cr['month_of_posting'], 1, 0)

In [ ]:
txn_dr_cr.head()

In [ ]:
# quick QC
txn_dr_cr.txn_cr_cnt.sum() + txn_dr_cr.txn_dr_cnt.sum() ,  txn_dr_cr.txn_abs_amt.sum().round() , txn_dr_cr.txn_tot_cnt.sum() 

In [ ]:
df = txn_dr_cr.pivot_table(index=['gl_account', 'month_of_posting'],
                                    columns='late_entry_flg', values=['txn_tot_cnt','txn_abs_amt'] ,  aggfunc=[np.sum]) # reshape data 
df = pd.DataFrame(df.to_records())
df.columns = (['gl_account', 'month_of_posting', 'txn_amt_bfr_closing',  'closing_txn_amt', 'txn_cnt_bfr_closing',  'closing_txn_cnt' ])
df = df.fillna(0)
df.tail()

In [ ]:
# quick QC
df.txn_cnt_bfr_closing.sum() + df.closing_txn_cnt.sum() , df.txn_amt_bfr_closing.sum() + df.closing_txn_amt.sum() 

# Calculate percentage variables

In [ ]:
df['txn_amt'] = df['txn_amt_bfr_closing']  + df['closing_txn_amt'] 
df['txn_amt'].head()

In [ ]:
df['txn_cnt'] = df['txn_cnt_bfr_closing']  + df['closing_txn_cnt'] 
df['txn_cnt'].head()

In [ ]:
df['closing_txns_amt_perc'] = df['closing_txn_amt'].div(df['txn_amt']).replace(np.inf, 0).round(2)
df['closing_txns_cnt_perc'] = df['closing_txn_cnt'].div(df['txn_cnt']).replace(np.inf, 0).round(2)

In [ ]:
df['closing_txns_cnt_perc'] .describe()

In [ ]:
df['closing_txns_amt_perc'] .describe()

In [ ]:
df.tail()

# QC of data

In [ ]:
# current data
df.txn_amt.sum(), df.txn_cnt.sum(), df.closing_txn_amt.sum(), df.closing_txn_cnt.sum()

In [ ]:
x = abs(data_txn[["Amt_loc_Currency"]]).sum()
y = abs(data_txn[["Amt_loc_Currency"]]).count()

print('%f' % x, y)

# Preprocessing - dates and gl accounts 

In [ ]:
# testing on a few transactions
transactions1 = df.loc[df["gl_account"] ==  '0009600016']
# transactions2 = df.loc[df["gl_account"] ==  '0067593000']
# transactions3 = df.loc[df["gl_account"] ==  '0069610030']
# transactions4 = df.loc[df["gl_account"] ==  '0060900000']
# transactions5 = df.loc[df["gl_account"] ==  '0048857020']
# df1 = transactions1.append(transactions2.append(transactions3.append(transactions4.append(transactions5))))
df1 = transactions1
# transactions1.shape, transactions2.shape, transactions3.shape, transactions4.shape, transactions5.shape, df1.shape

In [ ]:
df1.head()

In [ ]:
# df11 = txn_dr_cr.loc[txn_dr_cr["gl_account"] == "R1111300"]
# df11

In [ ]:
df1 = df[['gl_account', 'month_of_posting','closing_txn_amt', 'closing_txn_cnt', 'closing_txns_amt_perc', 'closing_txns_cnt_perc']]
df1.columns = ['gl_account', 'ds', 'closing_txn_amt', 'closing_txn_cnt', 'closing_txns_amt_perc', 'closing_txns_cnt_perc']
df1["ds"] = pd.to_datetime(df1["ds"])
df1.head()

# Exclusions

In [ ]:
# get account start date - date of first transaction
# get a cross tab of all dates for all GL accounts
# reduce the cross tab by taking dates only after first transaction
# run the model only on accounts with atleast 12 data points


### 1. Accounts newer than one year 

Get minimum txn date for each GL account

In [ ]:
df_txn_min_date = df1.loc[df1.groupby('gl_account')['ds'].idxmin()]
df_txn_min_date = df_txn_min_date[['gl_account', 'ds']]
df_txn_min_date.columns = ['gl_account', 'txn_min_date']
df_txn_min_date.head()

In [ ]:
df_txn_min_date.shape

In [ ]:
import dateutil
date_today = pd.to_datetime('today')
model_min_date = date_today - dateutil.relativedelta.relativedelta(months=12)
date_today, model_min_date

In [ ]:
df_txn_min_date1 = df_txn_min_date[df_txn_min_date['txn_min_date']<= model_min_date]
df_txn_min_date1.shape

In [ ]:
df_txn_min_date1.head()

### 2. Accounts with 2 or lesser late entries in two years

In [ ]:
df_max_closing_txn_cnt = df1.loc[df1.groupby('gl_account')['closing_txn_cnt'].idxmax()]
df_max_closing_txn_cnt = df_max_closing_txn_cnt[['gl_account', 'closing_txn_cnt']]
df_max_closing_txn_cnt.columns = ['gl_account', 'df_max_txn_cnt']
df_max_closing_txn_cnt.shape
# check if the amount in local currency for any of these accounts is > 50k 

In [ ]:
df_max_closing_txn_cnt

In [ ]:
df_max_closing_txn_cnt = df_max_closing_txn_cnt[(df_max_closing_txn_cnt['df_max_txn_cnt'] > 2)]
df_max_closing_txn_cnt.shape

### 3. Accounts starting wtih 4305 and 4306

In [ ]:
# 4305 or 4306
list_gl_ac = df1.gl_account.unique()
list_gl_ac_excl_4305_4306 = [x for x in list_gl_ac if not x.startswith(('004305', '004306'))]
len(list_gl_ac), len(list_gl_ac_excl_4305_4306)
# transactions1= transactions[transactions.gl_account.isin(list_gl_ac1)]


### Get final list of accounts for removal

In [ ]:
# df_AC_post_exclusions0 = data2

In [ ]:
df_AC_post_exclusions0 = df_max_closing_txn_cnt.merge(df_txn_min_date1, on = 'gl_account')
#df_max_closing_txn_cnt df_txn_min_date
df_AC_post_exclusions0.shape

In [ ]:
df_AC_post_exclusions = df_AC_post_exclusions0[df_AC_post_exclusions0.gl_account.isin(list_gl_ac_excl_4305_4306)]
df_AC_post_exclusions.shape

In [ ]:
df_AC_post_exclusions.head()

In [ ]:
df_chk = df_AC_post_exclusions.loc[df_AC_post_exclusions["gl_account"] ==  '0025397026']
df_chk

### Create empty frame of GL accounts * Month

In [ ]:
list_dates = pd.date_range(start="2018-10-01",end="2020-10-01",freq='M' ).tolist()
list_gl_ac_post_excl = df_AC_post_exclusions.gl_account.unique()
len(list_dates), len(list_gl_ac_post_excl)

In [ ]:
df_dates = pd.DataFrame({'date':list_dates})
df_list_gl_ac_post_excl = pd.DataFrame({'gl_account':list_gl_ac_post_excl})
df_list_gl_ac_post_excl['key'] = 0
df_dates['key'] = 0
dates_and_ac = df_list_gl_ac_post_excl.merge(df_dates, how='left', on = 'key')
dates_and_ac.drop('key',1, inplace=True)
dates_and_ac.shape

In [ ]:
### remove dates which fall before the minimum transaction dates for the accounts

In [ ]:
dates_and_ac.tail(5)

In [ ]:
dates_and_ac.shape

### Remove account*date combination occouring before the min txn date from empty rack

In [ ]:
dates_and_ac2 = pd.merge(dates_and_ac, df_AC_post_exclusions, left_on = ['gl_account'],  right_on = ['gl_account'], how = 'left')
dates_and_ac2.head(5)

In [ ]:
dates_and_ac3 = dates_and_ac2[(dates_and_ac2['txn_min_date'] <= dates_and_ac2['date'])]
dates_and_ac3.shape

In [ ]:
dates_and_ac3.head(5)

### Subset txn data post exclusions



In [ ]:
df1.shape

In [ ]:
df2 = pd.merge(dates_and_ac3[['gl_account', 'date']], df1, left_on = ['gl_account', 'date'],  right_on = ['gl_account', 'ds'], how = 'left')
df2.head()

In [ ]:
df2.shape

Merge the empty frame with main data to get y values

In [ ]:
num_cols = df2.select_dtypes(include=[np.number]).columns
df2[num_cols]=df2.select_dtypes(include=[np.number]).fillna(0)
df2.head()

In [ ]:
# remove GL accounts with less than 12 months of data 

In [ ]:
df2.to_csv('/home/ec2-user/SageMaker/data-science-ending-balance/code/data-science/03-modeling/model_data.csv',sep=",",index=False,encoding="utf-8")